# LDA Topic Modelling with GENSIM

In [12]:
import pandas as pd
import numpy as np
import json
import glob
import nltk

#Gensim libraries
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy libraries for lemmitisation
import spacy
from nltk.corpus import stopwords

#visualisations 
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore")

## Preparing the Data

In [13]:
def load_data(file):
    with open(file, "r", encoding = "utf-8") as f:
        data = json.load(f)
    return (data)

def write_data(file,data):
    with open(file, "w", enconding = "utf-8") as f:
        json.dump(data, f, indent = 4)

In [14]:
stopwords = stopwords.words("english")

In [15]:
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [19]:
data = pd.read_csv("threads_master_data.csv")["Post_Content"]

In [21]:
print(data[0][0:90])

In both the Wellingborough and Kingswood by-elections Reform UK achieved double digits. Th


In [24]:
def lemmatization(texts,allowed_postags = ["NOUN", "ADJ","VERB","ADV"]):
    try:
        nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    except OSError:
        # If model is not found, download it
        import subprocess
        subprocess.run(["python", "-m", "spacy", "download", "en_core_web_sm"])
        nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return(texts_out)

lemmatized_texts = lemmatization(data)
print(lemmatized_texts[0][0:90])

election achieve double digit squeeze put real expect continue tack right run general elec


In [27]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc = True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)
print(data_words[0][0:20])

['election', 'achieve', 'double', 'digit', 'squeeze', 'put', 'real', 'expect', 'continue', 'tack', 'right', 'run', 'general', 'election']


In [29]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)
    
print(corpus[0][0:20])

word = id2word[[0][:1][0]]
print(word)

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)]
achieve


In [30]:
lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus,
                                           id2word = id2word,
                                           num_topics = 30,
                                           random_state = 100,
                                           update_every = 1,
                                           chunksize = 100,
                                           alpha = "auto")

## Visualising the Data

In [32]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model,corpus,id2word,mds = "mmds", R = 30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.257869 -0.276093       1        1  11.989745
25     0.072712 -0.312749       2        1   7.775783
19    -0.359192 -0.077736       3        1   7.401692
4      0.241131 -0.192081       4        1   7.240019
28     0.268971 -0.116849       5        1   6.727126
21     0.320994 -0.004811       6        1   4.836501
8      0.297204  0.167938       7        1   4.614680
11     0.160654 -0.256344       8        1   4.480482
10    -0.216647 -0.050259       9        1   4.142510
12    -0.168880 -0.170279      10        1   3.685066
27    -0.265160  0.085497      11        1   3.426445
20    -0.261445  0.182458      12        1   3.262188
6      0.040610 -0.147563      13        1   3.256444
14    -0.058193 -0.194308      14        1   2.943072
22     0.055786  0.290837      15        1   2.374379
2     -0.131782  0.261140      16        1   2.373434
15    -0.086740  0.159526      17        1   2.153787
3     -0.079656 -0.041780      18        1   2.142801
5     -0.048086 -0.320272      19        1   1.809610
16     0.074963 -0.021296      20        1   1.746646
29     0.170534  0.060511      21        1   1.609228
24     0.097035  0.174918      22        1   1.542641
17    -0.134814  0.057221      23        1   1.486748
23    -0.003622  0.005575      24        1   1.471679
0     -0.012015  0.187146      25        1   1.463134
7      0.038448  0.181629      26        1   1.077666
18     0.149861  0.114691      27        1   0.990511
13     0.052620  0.099534      28        1   0.906095
1      0.033603  0.066599      29        1   0.623898
26     0.008977  0.087199      30        1   0.445988, topic_info=         Term        Freq       Total Category  logprob  loglift
61        win   33.000000   33.000000  Default  30.0000  30.0000
4    election  520.000000  520.000000  Default  29.0000  29.0000
378      week   34.000000   34.000000  Default  28.0000  28.0000
9       right   81.000000   81.000000  Default  27.0000  27.0000
106     party   81.000000   81.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
16     author    0.013872    1.081272  Topic30  -7.6825   1.0566
17        com    0.013872    6.951101  Topic30  -7.6825  -0.8041
18      cover    0.013872    2.353576  Topic30  -7.6825   0.2788
19    excited    0.013872    1.250584  Topic30  -7.6825   0.9112
20     google    0.013872    1.081255  Topic30  -7.6825   1.0566

[1304 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
137       4  0.460840       able
137       5  0.506924       able
254       1  0.876925      about
351      19  0.755403      above
118       5  0.855738   absolute
...     ...       ...        ...
178      13  0.177486       year
178      21  0.050710       year
554      12  0.613911  yesterday
528      10  0.534884        yet
117      20  0.662024      young

[881 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 26, 20, 5, 29, 22, 9, 12, 11, 13, 28, 21, 7, 15, 23, 3, 16, 4, 6, 17, 30, 25, 18, 24, 1, 8, 19, 14, 2, 27])